# Es un perro o un gato?


![](https://imagenes.elpais.com/resizer/3Te8-EEwo6Q2IhPOOLZe5KGu-78=/1200x0/cloudfront-eu-central-1.images.arcpublishing.com/prisa/AMH2ADNVDVDZBOLV2UVWWV4I4E.jpg)

# Convolutional Neural Network

En el aprendizaje profundo, una red neuronal convolucional (CNN, por sus siglas en inglés) es una clase de redes neuronales profundas que se aplican comúnmente al análisis de imágenes visuales. También se conocen como redes neuronales artificiales con invariantes de desplazamiento o invariantes de espacio (SIANN, por sus siglas en inglés) debido a su arquitectura de pesos compartidos y sus características de invarianza a la translación. Tienen aplicaciones en el reconocimiento de imágenes y videos, sistemas de recomendación, clasificación de imágenes, análisis de imágenes médicas, procesamiento de lenguaje natural y series temporales financieras.

Las CNN son versiones regularizadas de perceptrones multicapa. Los perceptrones multicapa suelen referirse a redes completamente conectadas, es decir, cada neurona en una capa está conectada a todas las neuronas en la capa siguiente. La "total conectividad" de estas redes las hace propensas al sobreajuste de los datos. Las formas típicas de regularización incluyen agregar alguna forma de medida de la magnitud de los pesos a la función de pérdida. Las CNN adoptan un enfoque diferente hacia la regularización: aprovechan el patrón jerárquico en los datos y ensamblan patrones más complejos utilizando patrones más pequeños y simples. Por lo tanto, en términos de conectividad y complejidad, las CNN se encuentran en el extremo inferior.

Las redes convolucionales se inspiraron en procesos biológicos, ya que el patrón de conectividad entre las neuronas se asemeja a la organización de la corteza visual de los animales. Las neuronas corticales individuales responden a estímulos solo en una región restringida del campo visual conocida como campo receptivo. Los campos receptivos de diferentes neuronas se superponen parcialmente de manera que cubren todo el campo visual.

Las CNN utilizan relativamente poco preprocesamiento en comparación con otros algoritmos de clasificación de imágenes. Esto significa que la red aprende los filtros que en los algoritmos tradicionales se diseñaban manualmente. Esta independencia del conocimiento previo y del esfuerzo humano en el diseño de características es una ventaja importante.

# Arquitectura de una CNN

Una red neuronal convolucional consta de una capa de entrada, una capa de salida y múltiples capas ocultas. Las capas ocultas de una CNN suelen estar compuestas por una serie de capas de convolución que convolucionan mediante una multiplicación u otro producto escalar. La función de activación suele ser una capa RELU, seguida posteriormente por capas adicionales de convolución, como capas de agrupación, capas completamente conectadas y capas de normalización, conocidas como capas ocultas porque sus entradas y salidas están enmascaradas por la función de activación y la convolución final.

Aunque coloquialmente se les llama "capas de convolución", esto es solo por convención. Matemáticamente, técnicamente es un producto escalar deslizante o una correlación cruzada. Esto tiene importancia para los índices en la matriz, ya que afecta cómo se determina el peso en un punto de índice específico.

![](https://miro.medium.com/max/1866/0*AONVmd3v4wO_dWr6)

In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf

In [2]:
from keras.preprocessing.image import ImageDataGenerator

# Preprocesamiento de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Escala los valores de píxeles al rango [0, 1]
    shear_range=0.2,  # Rango de cambio de inclinación de la imagen
    zoom_range=0.2,   # Rango de zoom de la imagen
    horizontal_flip=True  # Voltea horizontalmente algunas imágenes
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Generando el conjunto de entrenamiento
training_set = train_datagen.flow_from_directory(
    '/kaggle/input/dogs-vs-cats-train-validadion-and-evaluation/train/',  # Directorio de las imágenes de entrenamiento
    target_size=(64, 64),  # Redimensiona las imágenes a 64x64 píxeles
    batch_size=32,  # Tamaño del lote de imágenes
    class_mode='binary'  # Problema de clasificación binaria
)

# Generando el conjunto de prueba
test_set = test_datagen.flow_from_directory(
    '/kaggle/input/dogs-vs-cats-train-validadion-and-evaluation/data/validation/',  # Directorio de las imágenes de prueba
    target_size=(64, 64),  # Redimensiona las imágenes a 64x64 píxeles
    batch_size=32,  # Tamaño del lote de imágenes
    class_mode='binary'  # Problema de clasificación binaria
)


Using TensorFlow backend.


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


## Convolución
Cuando se programa una CNN, la entrada es un tensor con forma (número de imágenes) x (altura de la imagen) x (ancho de la imagen) x (profundidad de la imagen). Luego, después de pasar por una capa convolucional, la imagen se abstrae a un mapa de características, con forma (número de imágenes) x (altura del mapa de características) x (ancho del mapa de características) x (canales del mapa de características). Una capa convolucional en una red neuronal debe tener los siguientes atributos:

* Kernels de convolución definidos por un ancho y una altura (hiperparámetros).
* El número de canales de entrada y canales de salida (hiperparámetro).
* La profundidad del filtro de convolución (los canales de entrada) debe ser igual al número de canales (profundidad) del mapa de características de entrada.

Las capas convolucionales convolucionan la entrada y pasan su resultado a la siguiente capa. Esto es similar a la respuesta de una neurona en la corteza visual a un estímulo específico. Cada neurona convolucional procesa datos solo para su campo receptivo. Aunque las redes neuronales totalmente conectadas de avance pueden usarse para aprender características y clasificar datos, no es práctico aplicar esta arquitectura a imágenes. Sería necesario un número muy alto de neuronas, incluso en una arquitectura poco profunda, debido a los tamaños de entrada muy grandes asociados con las imágenes, donde cada píxel es una variable relevante. Por ejemplo, una capa completamente conectada para una imagen (pequeña) de tamaño 100 x 100 tiene 10,000 pesos para cada neurona en la segunda capa. La operación de convolución resuelve este problema al reducir el número de parámetros libres, lo que permite que la red sea más profunda con menos parámetros. Por ejemplo, independientemente del tamaño de la imagen, regiones de mosaico de tamaño 5 x 5, cada una con los mismos pesos compartidos, requieren solo 25 parámetros aprendibles. Al usar pesos regularizados sobre menos parámetros, se evitan los problemas de desvanecimiento del gradiente y explosión del gradiente que se observan durante la retropropagación en las redes neuronales tradicionales.

## Pooling
Las redes convolucionales pueden incluir capas de agrupación locales o globales para simplificar la computación subyacente. Las capas de agrupación reducen las dimensiones de los datos al combinar las salidas de grupos de neuronas en una capa en una sola neurona en la siguiente capa. La agrupación local combina pequeños grupos, generalmente de tamaño 2x2. La agrupación global actúa sobre todas las neuronas de la capa de convolución. Además, la agrupación puede calcular el máximo o el promedio. La agrupación máxima utiliza el valor máximo de cada grupo de neuronas en la capa anterior. La agrupación promedio utiliza el valor promedio de cada grupo de neuronas en la capa anterior.

## Campo receptivo
En las redes neuronales, cada neurona recibe entrada desde cierto número de ubicaciones en la capa anterior. En una capa completamente conectada, cada neurona recibe entrada de cada elemento de la capa anterior. En una capa convolucional, las neuronas reciben entrada solo de un área restringida de la capa anterior. Típicamente, el área restringida tiene forma cuadrada (por ejemplo, de tamaño 5 por 5). El área de entrada de una neurona se llama su campo receptivo. Entonces, en una capa completamente conectada, el campo receptivo es toda la capa anterior. En una capa convolucional, el área receptiva es más pequeña que toda la capa anterior. El subárea de la imagen de entrada original en el campo receptivo crece cada vez más a medida que se profundiza en la arquitectura de la red. Esto se debe a que se aplica una y otra vez una convolución que tiene en cuenta el valor de un píxel específico, pero también algunos píxeles circundantes.

## Pesos
Cada neurona en una red neuronal calcula un valor de salida aplicando una función específica a los valores de entrada que provienen del campo receptivo en la capa anterior. La función que se aplica a los valores de entrada está determinada por un vector de pesos y un sesgo (típicamente números reales). El aprendizaje en una red neuronal avanza mediante ajustes iterativos de estos sesgos y pesos.

El vector de pesos y el sesgo se llaman filtros y representan características particulares de la entrada (por ejemplo, una forma específica). Una característica distintiva de las CNN es que muchas neuronas pueden compartir el mismo filtro. Esto reduce la huella de memoria porque se utiliza un solo sesgo y un solo vector de pesos en todos los campos receptivos que comparten ese filtro, en lugar de que cada campo receptivo tenga su propio sesgo y vector de ponderación.

![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS9IV64_W5igS-vk8C9LV1NIPyl-u1eQOrRKA&usqp=CAU)

In [3]:
# Parte 2 - Construyendo la CNN

# Inicializando la red neuronal convolucional (CNN)
cnn = tf.keras.models.Sequential()

# Paso 1 - Convolución
cnn.add(tf.keras.layers.Conv2D(
    filters=32,             # Número de filtros o características en la capa
    kernel_size=3,          # Tamaño del kernel (ventana) de convolución 3x3
    padding="same",         # Relleno para mantener la misma forma de la imagen
    activation="relu",      # Función de activación ReLU (Rectified Linear Unit)
    input_shape=[64, 64, 3] # Formato de entrada de las imágenes
))

# Paso 2 - Agrupación
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,            # Tamaño de la ventana de agrupación 2x2
    strides=2,              # Desplazamiento (stride) de la ventana
    padding='valid'         # Sin relleno, puede descartar información en los bordes
))

# Agregando una segunda capa de convolución
cnn.add(tf.keras.layers.Conv2D(
    filters=32,             # Número de filtros
    kernel_size=3,          # Tamaño del kernel 3x3
    padding="same",         # Relleno "same" para mantener la misma forma
    activation="relu"       # Función de activación ReLU
))
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,            # Tamaño de la ventana de agrupación 2x2
    strides=2,              # Desplazamiento de la ventana
    padding='valid'         # Sin relleno
))

cnn.add(tf.keras.layers.Conv2D(
    filters=32,             # Número de filtros
    kernel_size=3,          # Tamaño del kernel 3x3
    padding="same",         # Relleno "same"
    activation="relu"       # Función de activación ReLU
))
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,            # Tamaño de la ventana de agrupación 2x2
    strides=2,              # Desplazamiento de la ventana
    padding='valid'         # Sin relleno
))

cnn.add(tf.keras.layers.Conv2D(
    filters=32,             # Número de filtros
    kernel_size=3,          # Tamaño del kernel 3x3
    padding="same",         # Relleno "same"
    activation="relu"       # Función de activación ReLU
))
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,            # Tamaño de la ventana de agrupación 2x2
    strides=2,              # Desplazamiento de la ventana
    padding='valid'         # Sin relleno
))

# Paso 3 - Aplanamiento
cnn.add(tf.keras.layers.Flatten())

# Paso 4 - Conexión Completa
cnn.add(tf.keras.layers.Dense(
    units=128,              # Número de unidades en la capa oculta
    activation='relu'       # Función de activación ReLU
))

# Paso 5 - Capa de Salida
cnn.add(tf.keras.layers.Dense(
    units=1,                # Una unidad para la clasificación binaria
    activation='sigmoid'    # Función de activación sigmoide para la salida binaria
))


### Understanding convolutions

![](https://miro.medium.com/max/1070/1*Zx-ZMLKab7VOCQTxdZ1OAw.gif)

![](https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2018/04/06111147/do-dogs-hate-cats.jpg)

In [4]:
# Parte 3 - Entrenando la CNN

# Compilando la CNN
cnn.compile(
    optimizer='adam',                    # Algoritmo de optimización (Adam)
    loss='binary_crossentropy',          # Función de pérdida (entropía cruzada binaria)
    metrics=['accuracy']                # Métricas a seguir durante el entrenamiento (precisión)
)

# Entrenando la CNN en el conjunto de entrenamiento y evaluándola en el conjunto de prueba
for i in range(0, 2):  # Se realiza el entrenamiento dos veces (dos iteraciones)
    cnn.fit_generator(
        training_set,                    # Conjunto de entrenamiento
        steps_per_epoch=10,             # Número de pasos por epoch (lotes por epoch)
        epochs=10,                       # Número de epochs (iteraciones completas a través de los datos)
        validation_data=test_set,        # Conjunto de prueba para la validación
        validation_steps=60             # Número de pasos de validación (lotes de validación por epoch)
    )


Epoch 1/10
10/10 [==============================] - 23s 2s/step - loss: 0.7034 - accuracy: 0.5125 - val_loss: 0.6931 - val_accuracy: 0.4880
Epoch 2/10
10/10 [==============================] - 21s 2s/step - loss: 0.6928 - accuracy: 0.5437 - val_loss: 0.6935 - val_accuracy: 0.4969
Epoch 3/10
10/10 [==============================] - 20s 2s/step - loss: 0.6907 - accuracy: 0.5375 - val_loss: 0.6948 - val_accuracy: 0.4995
Epoch 4/10
10/10 [==============================] - 19s 2s/step - loss: 0.6901 - accuracy: 0.5188 - val_loss: 0.6880 - val_accuracy: 0.5542
Epoch 5/10
10/10 [==============================] - 19s 2s/step - loss: 0.6854 - accuracy: 0.5594 - val_loss: 0.6896 - val_accuracy: 0.5141
Epoch 6/10
10/10 [==============================] - 18s 2s/step - loss: 0.6813 - accuracy: 0.5625 - val_loss: 0.6780 - val_accuracy: 0.5729
Epoch 7/10
10/10 [==============================] - 18s 2s/step - loss: 0.6869 - accuracy: 0.5437 - val_loss: 0.6725 - val_accuracy: 0.5911
Epoch 8/10
10/10 [==

![](https://miro.medium.com/max/3840/1*oB3S5yHHhvougJkPXuc8og.gif)

In [5]:
# Evaluar el modelo en los datos de prueba utilizando `evaluate`
print("Evaluar en los datos de prueba")
results = cnn.evaluate(test_set, batch_size=128)
print("pérdida de prueba, precisión de prueba:", results)

Evaluar en los datos de prueba
157/157 [==============================] - 37s 238ms/step - loss: 0.6758 - accuracy: 0.5750
pérdida de prueba, precisión de prueba: [0.6758036613464355, 0.574999988079071]
